# BERTのクラス分類で役職を推定するモデルを構成し、学習させる

## 1. モジュールのインポート

In [1]:
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

import pickle

In [2]:
from aiwolf import Role,Agent

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

## モデルの定義

In [4]:
# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
MAX_LENGTH = 512
BATCH_SIZE = 128
# 文章をトークンに変換するトークナイザーの読み込み
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

In [5]:
class BertForSequenceClassification_pl(pl.LightningModule):
    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        
    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss
        
    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる。

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels') # バッチからラベルを取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) #精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

## 前処理

In [6]:
dataset_for_loader = []

# データを取得
#現在のディレクトリを取得
import pathlib
from pathlib import Path

current_dir = pathlib.Path().resolve()
data_set_plain = pickle.load(open(current_dir.joinpath("preprocessed_data").joinpath('dataset.pkl'), 'rb'))
labels_list = [Role.VILLAGER,Role.SEER,Role.BODYGUARD,Role.MEDIUM,Role.WEREWOLF,Role.POSSESSED,Role.FOX,Role.FREEMASON]

for data in data_set_plain:
    encoding = tokenizer(
            data[1],
            max_length=MAX_LENGTH, 
            padding='max_length',
            truncation=True
        )
    
    encoding['labels'] = labels_list.index(Role(data[0]))
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)

In [7]:
# データセットの分割
random.shuffle(dataset_for_loader) # ランダムにシャッフル
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:] # テストデータ

# データセットからデータローダを作成
# 学習データはshuffle=Trueにする。
dataloader_train = DataLoader(
    dataset_train, batch_size=BATCH_SIZE//4, shuffle=True
) 
dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE)

## 学習

In [8]:
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=3,
    save_weights_only=True,
    dirpath='bert_role_estimation_model/',
)

# 学習の方法を指定
trainer = pl.Trainer(
    max_epochs=10,
    callbacks = [checkpoint]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/robot_dev6/okubo/aiwolf/AIWolfK2B/k2b_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


In [9]:
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=len(labels_list), lr=1e-5
)

# ファインチューニングを行う。
trainer.fit(model, dataloader_train, dataloader_val) 

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Sanity Checking: 0it [00:00, ?it/s]

/home/robot_dev6/okubo/aiwolf/AIWolfK2B/k2b_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/robot_dev6/okubo/aiwolf/AIWolfK2B/k2b_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [10]:
# 検証データで確認
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

ベストモデルのファイル:  /storage/home/robot_dev6/okubo/aiwolf/AIWolfK2B/aiwolfk2b/AttentionReasoningAgent/Modules/bert_role_estimation_model/epoch=9-step=11530.ckpt
ベストモデルの検証データに対する損失:  tensor(0.3224, device='cuda:0')


In [11]:
# テストデータで確認
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

/home/robot_dev6/okubo/aiwolf/AIWolfK2B/k2b_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at /storage/home/robot_dev6/okubo/aiwolf/AIWolfK2B/aiwolfk2b/AttentionReasoningAgent/Modules/bert_role_estimation_model/epoch=9-step=11530.ckpt
LOCAL_RANK

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy            0.8543768525123596
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Accuracy: 0.85


In [12]:
# Transformers対応のモデルを保存
out_model_dir = './bert_role_estimation_model'

import datetime
today = datetime.date.today()
out_filename = format(today, '%Y%m%d')

torch.save(model.bert_sc.state_dict(),out_model_dir + '/' + f'{out_filename}.pth')
model.bert_sc.save_pretrained('./bert_role_estimation_model') 

In [15]:
# モデルのロード
best_model_path = "./bert_role_estimation_model/20230625.pth"
bert_sc_best = BertForSequenceClassification.from_pretrained(
            MODEL_NAME,
            num_labels=len(labels_list),
        )
bert_sc_best.load_state_dict(torch.load(best_model_path))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

<All keys matched successfully>

In [16]:
#保存したモデルが取得できるか確認
bert_sc = BertForSequenceClassification.from_pretrained(
    './bert_role_estimation_model'
)
bert_sc = bert_sc.cuda()

In [18]:
#学習したモデルの検証
test_input = ["""4,1,0,0,1,1,1,0
SEER
day1
talk:
もはもは＾－＾
もはもは＾－＾
もは＾－＾占いCO[02]たん◯れした＾－＾
もは＾－＾
もは＾－＾寒い所では…花は枯れてしまうの…；－；
もはよう＾－＾
占い把握＾－＾
[03]たん占い把握＾－＾
占いco[08]○
占い把握＾－＾
占い2把握＾－＾
[04]たんも占いね＾－＾
占い2把握ら＾－＾真狂か真狐とかかのあ＾－＾
人外全潜伏かしら＾－＾狂人はいなさそう？＾－＾
対抗把握しマス＾－＾
役職欠けて狂狐の可能性も＾－＾
>>13狂いない可能性もある＾－＾
対抗把握＾－＾狂>狼狐かな＾－＾
占い2出てるから全潜伏れはないれそ＾－＾真狂めかのあ＾－＾
[04]たんちょっと出方様子見っぽく思えたから真目下がるのあ；－；
狂人いなくて占い欠けの狼狐らったらやばえ；－；
>>17おんその場合は狼か狐が出てるのもあるなって＾－＾
とりま占い先宣言してほしいお＾－＾
今日はグレラン？＾－＾
グレーから柱出てもらう？＾－＾吊りあんもしゆゆうのいけお＾－＾
呪殺ないと真なのかまからん；－；
吊りは狐先に吊らないとら＾－＾
じゃあ[07]たん占う＾－＾
宣言したほうがいいかんじ？＾－＾
吊り余裕は銃殺出してもらえば増えるし対抗占いしてもらいたいかも＾－＾
じゃあ[06]たん行きます
漏れ吊っていいお＾－＾
day2
divine,1,4,HUMAN
talk:""","""4,1,0,0,1,1,1,0
VILLAGER
day1
talk:
もはもは＾－＾
もはもは＾－＾
もは＾－＾占いCO[01]たん◯れした＾－＾
もは＾－＾
もは＾－＾寒い所では…花は枯れてしまうの…；－；
もはよう＾－＾
占い把握＾－＾
[02]たん占い把握＾－＾
占いco[07]○
占い把握＾－＾
占い2把握＾－＾
[03]たんも占いね＾－＾
占い2把握ら＾－＾真狂か真狐とかかのあ＾－＾
人外全潜伏かしら＾－＾狂人はいなさそう？＾－＾
対抗把握しマス＾－＾
役職欠けて狂狐の可能性も＾－＾
>>13狂いない可能性もある＾－＾
対抗把握＾－＾狂>狼狐かな＾－＾
占い2出てるから全潜伏れはないれそ＾－＾真狂めかのあ＾－＾
[03]たんちょっと出方様子見っぽく思えたから真目下がるのあ；－；
狂人いなくて占い欠けの狼狐らったらやばえ；－；
>>17おんその場合は狼か狐が出てるのもあるなって＾－＾
とりま占い先宣言してほしいお＾－＾
今日はグレラン？＾－＾
グレーから柱出てもらう？＾－＾吊りあんもしゆゆうのいけお＾－＾
呪殺ないと真なのかまからん；－；
吊りは狐先に吊らないとら＾－＾
じゃあ[06]たん占う＾－＾
宣言したほうがいいかんじ？＾－＾
吊り余裕は銃殺出してもらえば増えるし対抗占いしてもらいたいかも＾－＾
じゃあ[05]たん行きます
漏れ吊っていいお＾－＾
"""]

truth_label = [Role.WEREWOLF, Role.VILLAGER]

encoding = tokenizer(
        test_input,
        max_length=MAX_LENGTH, 
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
encoding = { k: v.cuda() for k, v in encoding.items() }

# 推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits # 分類スコア
labels_predicted = scores.argmax(-1).cpu().numpy() # スコアが最も高いラベル
labels_predicted = [labels_list[i] for i in labels_predicted] # ラベルをラベル名に変換
print(labels_predicted)
    

[<Role.WEREWOLF: 'WEREWOLF'>, <Role.VILLAGER: 'VILLAGER'>]
